# Introduction

In this lecture we will use the MovieLens dataset containing 100k movie ratings.

MovieLens site: https://grouplens.org/datasets/movielens/

100k dataset: http://files.grouplens.org/datasets/movielens/ml-100k.zip


## Setting up Hadoop and Hive envvars

In [ ]:
%env HADOOP_VERSION     2.9.2
%env HADOOP_HOME hadoop-2.9.2

In [ ]:
%env HIVE_VERSION     hive-2.3.5
%env HIVE_HOME apache-hive-2.3.5-bin

## Setting up Sqoop envvars

In [ ]:
%env SQOOP_VERSION     1.4.7
%env SQOOP_HOME sqoop-1.4.7.bin__hadoop-2.6.0

## Java Home envvar

In [ ]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64
# !echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " >> ~/.bashrc
# !echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " >> ~/.profile

In [ ]:
#!cp $(pwd)/${SQOOP_HOME}/conf/sqoop-env-template.sh $(pwd)/${SQOOP_HOME}/conf/sqoop-env.sh
#!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> $(pwd)/${HIVE_HOME}/bin/sqoop-env.sh
!echo "export HADOOP_COMMON_HOME=$(pwd)/$HADOOP_HOME"             >> $(pwd)/${SQOOP_HOME}/conf/sqoop-env.sh
!echo "export HADOOP_MAPRED_HOME=$(pwd)/$HADOOP_HOME"             >> $(pwd)/${SQOOP_HOME}/conf/sqoop-env.sh
!echo "export HIVE_HOME=$(pwd)/$HIVE_HOME"                 >> $(pwd)/${SQOOP_HOME}/conf/sqoop-env.sh

## Setting the Sqoop path envvar

In [ ]:
!echo "export SQOOP_HOME=$(pwd)/$SQOOP_HOME" >> ~/.bashrc
!echo "export SQOOP_HOME=$(pwd)/$SQOOP_HOME" >> ~/.profile

## Setting the Hive path envvar

In [ ]:
!echo "export HIVE_HOME=$(pwd)/$HIVE_HOME" >> ~/.bashrc
!echo "export HIVE_HOME=$(pwd)/$HIVE_HOME" >> ~/.profile

In [ ]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> $(pwd)/${HIVE_HOME}/bin/hive-env.sh
!echo "export HADOOP_HOME=$(pwd)/$HADOOP_HOME"             >> $(pwd)/${HIVE_HOME}/bin/hive-env.sh
!echo "export HIVE_HOME=$(pwd)/$HIVE_HOME"                 >> $(pwd)/${HIVE_HOME}/bin/hive-env.sh

# Creating a database in SQLite3

In [ ]:
#Verifying created databases
!sqlite3 test.db "create table people (id int, name text);" ".exit"
!sqlite3 test.db "INSERT INTO people values(1,'Theo');" "INSERT INTO people values(2,'Bug');" ".exit"
!sqlite3 test.db "SELECT * FROM people;" ".exit"



#Creating a database
#!./$HIVE_HOME/bin/beeline -u "jdbc:hive2://" -e "CREATE DATABASE IF NOT EXISTS movielens;"

#!./$HIVE_HOME/bin/beeline -u "jdbc:hive2://" -e "SHOW DATABASES;"


In [ ]:
#!wget https://bitbucket.org/xerial/sqlite-jdbc/downloads/sqlite-jdbc-3.27.2.1.jar -q --show-progress

In [ ]:
#!echo "export HADOOP_CLASSPATH=$HADOOP_CLASSPATH:$(pwd)/sqlite-jdbc-3.27.2.1.jar"             >> $(pwd)/${HADOOP_HOME}/etc/hadoop/hadoop-env.sh

In [ ]:
#!cp sqlite-jdbc-3.27.2.1.jar $(pwd)/$SQOOP_HOME/lib

# Export data using sqoop

In [ ]:
!$SQOOP_HOME/bin/sqoop import --connect jdbc:sqlite://localhost/test.db --table people --m 1 --target-dir /user --driver org.sqlite.JDBC

## Creating a table ratings in the movielens database

```bash
USE movielens;

# Verifying tables
SHOW tables;

#Creating table ratings
CREATE TABLE ratings(
        userID  INT,
        movieID INT,
        rating  INT,
        time    INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE;

SHOW tables;
```

## Import data from local file system

```bash
SELECT * FROM ratings;

#Loading data
LOAD DATA LOCAL INPATH 'resources/examples/u.data'
OVERWRITE INTO TABLE ratings;

SELECT * FROM ratings LIMIT 10;
```

## Verifying the file created by Hive

In [ ]:
%env HADOOP_VERSION     2.9.2
%env HADOOP_HOME hadoop-2.9.2

!./$HADOOP_HOME/bin/hdfs dfs -ls /user/hive/warehouse/movielens.db/ratings/

## Finding the most popular movie

```bash
SELECT movieID, COUNT(movieID) as ratingCount
FROM ratings
GROUP BY movieID
ORDER BY ratingCount DESC
LIMIT 10;
```

## Finding the name of the most popular movie

### Creating a new table that contains movies' title

```bash
#Creating a new table called movieNames
CREATE TABLE movieNames(
        movieID  INT,
        title STRING)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '|'
STORED AS TEXTFILE;

#Loading data into movieNames table
LOAD DATA LOCAL INPATH 'resources/examples/u.item'
OVERWRITE INTO TABLE movieNames;
```

### Creating a view to store the movies' popularity

```bash
CREATE VIEW topMoviesIds AS
SELECT movieID, COUNT(movieID) as ratingCount
FROM ratings
GROUP BY movieID
ORDER BY ratingCount DESC;
```

### Finding the name of the most popular movie

```bash
SELECT n.title, ratingCount
FROM topMoviesIds t JOIN movieNames n ON t.movieID = n.movieID
LIMIT 10;
```

# It's your turn

## Create new tables and import the u.data and u.item using the HDFS

## Using your tables:

### Find the movie with the highest average time

What do you think about this result??

### Find the movie with the highest average time, only considering movies with more than 10 ratings